In [8]:
import pandas as pd
import numpy as np

In [2]:
path = "../capture24"

In [4]:
def create_sample_datasets(lower_bound, upper_bound, dataset_name, path):
    df_all = pd.DataFrame({'x': [], 'y': [], 'z': [], 'user_id': []})  # add a 'user_id' column to the DataFrame
    for i in range(lower_bound, upper_bound):

        user_id = f'P{i+1:03d}'  # set the user ID for this iteration

        df = pd.read_csv(f'{path}/{user_id}.csv.gz', compression='gzip', low_memory=False)
        df = df.drop(columns=['annotation'])
        
        # convert the 'time' column to a datetime object
        df['time'] = pd.to_datetime(df['time'], origin='unix')
        
        # set the 'time' column as the DataFrame index
        df.set_index('time', inplace=True)
        
        # resample the DataFrame from 100 Hz to 50 Hz
        df = df.resample('20ms').mean()
        
        df = df.reset_index().reset_index(drop=True)
        df['user_id'] = i+1  # set the user ID for this DataFrame
        df_all = pd.concat([df_all, df.loc[:df.shape[0] - df.shape[0] % 100 - 1]])
        print(f'{user_id}: {df.shape[0]}')
        del df

    # reset the index of the df_all DataFrame
    df_all = df_all.reset_index(drop=True)

    # save the df_all DataFrame to a Feather file
    df_all.to_feather(f'{path}/{dataset_name}.feather')
    del df_all

In [5]:
create_sample_datasets(0, 20, 'training', path)
create_sample_datasets(20, 22, 'validation', path)
create_sample_datasets(22, 24, 'test', path)

P021: 4770001
P022: 5340001
P023: 5040001
P024: 5040001
